In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
print(tf.__version__)


2.3.1


In [2]:

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


In [3]:

import glob
import ntpath
import cv2


In [4]:

from sklearn.metrics.pairwise import cosine_similarity
import scipy as sc

In [26]:

image_paths = glob.glob('image/*.jpg')
print(f'Found [{len(image_paths)}] images')

images = {}
for image_path in image_paths:
    image = cv2.imread(image_path, 3)
    b,g,r = cv2.split(image)           # get b, g, r
    image = cv2.merge([r,g,b])         # switch it to r, g, b
    image = cv2.resize(image, (200, 200))
    images[ntpath.basename(image_path)] = image      

n_col = 8
n_row = int(len(images)/n_col)
f, ax = plt.subplots(n_row, n_col, figsize=(16, 8))
for i in range(n_row):
    for j in range(n_col):
        ax[i, j].imshow(list(images.values())[n_col*i + j])
        ax[i, j].set_axis_off()

Found [0] images


<Figure size 1152x576 with 0 Axes>

In [8]:
def load_image(image):
  image = plt.imread(image)
  img = tf.image.convert_image_dtype(image, tf.float32)
  img = tf.image.resize(img, [400, 400])
  img = img[tf.newaxis, :] # shape -> (batch_size, h, w, d)
  return img

# content layers describe the image subject
content_layers = ['block5_conv2'] 

# style layers describe the image style
# we exclude the upper level layes to focus on small-size style details
style_layers = [ 
        'block1_conv1',
        'block2_conv1',
        'block3_conv1', 
        #'block4_conv1', 
        #'block5_conv1'
    ] 

def selected_layers_model(layer_names, baseline_model):
  outputs = [baseline_model.get_layer(name).output for name in layer_names]
  model = Model([vgg.input], outputs)
  return model

# style embedding is computed as concatenation of gram matrices of the style layers
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

class StyleModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleModel, self).__init__()
    self.vgg =  selected_layers_model(style_layers + content_layers, vgg)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    # scale back the pixel values
    inputs = inputs*255.0
    # preprocess them with respect to VGG19 stats
    preprocessed_input = preprocess_input(inputs)
    # pass through the reduced network
    outputs = self.vgg(preprocessed_input)
    # segregate the style and content representations
    style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                      outputs[self.num_style_layers:])

    # calculate the gram matrix for each layer
    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    # assign the content representation and gram matrix in
    # a layer by layer fashion in dicts
    content_dict = {content_name:value
                    for content_name, value
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}

    return {'content':content_dict, 'style':style_dict}

vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

def image_to_style(image_tensor):
    extractor = StyleModel(style_layers, content_layers)
    return extractor(image_tensor)['style']

def style_to_vec(style):
    # concatenate gram matrics in a flat vector
    return np.hstack([np.ravel(s) for s in style.values()]) 
    
# compute styles
image_style_embeddings = {}
for image_path in tqdm(image_paths): 
    image_tensor = load_image(image_path)
    style = style_to_vec( image_to_style(image_tensor) )
    image_style_embeddings[ntpath.basename(image_path)] = style


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [01:24<00:00,  2.64s/it]


In [9]:
def search_by_style(reference_image, max_results=10):
    v0 = image_style_embeddings[reference_image]
    distances = {}
    for k,v in image_style_embeddings.items():
        d = sc.spatial.distance.cosine(v0, v)
        distances[k] = d

    sorted_neighbors = sorted(distances.items(), key=lambda x: x[1], reverse=False)
    return sorted_neighbors[:11]
    
#     f, ax = plt.subplots(1, max_results, figsize=(16, 8))
#     for i, img in enumerate(sorted_neighbors[:max_results]):
#         ax[i].imshow(images[img[0]])
#         ax[i].set_axis_off()
    
#     plt.show()

In [21]:
search_by_style("s_cubism-05.jpg")

[('s_cubism-05.jpg', 0.0),
 ('s_impressionist-13.jpg', 0.07504183053970337),
 ('s_cubism-11.jpg', 0.07880634069442749),
 ('s_cubism-16.jpg', 0.08178341388702393),
 ('s_cubism-01.jpg', 0.085743248462677),
 ('s_cubism-08.jpg', 0.09565407037734985),
 ('s_cubism-09.jpg', 0.10248124599456787),
 ('s_impressionist-09.jpg', 0.10249888896942139),
 ('s_cubism-14.jpg', 0.10747575759887695),
 ('s_cubism-03.jpg', 0.10908150672912598),
 ('s_cubism-04.jpg', 0.10984539985656738)]

In [11]:
imagenames = [search_by_style(image_name) for image_name in images.keys()]

In [12]:
imagename = pd.DataFrame(imagenames)
imagename

,0,1,2,3,4,5,6,7,8,9,10
0,"(s_cubism-01.jpg, 0.0)","(s_impressionist-09.jpg, 0.05311310291290283)","(s_impressionist-12.jpg, 0.08358842134475708)","(s_cubism-05.jpg, 0.085743248462677)","(s_cubism-12.jpg, 0.08878123760223389)","(s_cubism-11.jpg, 0.09850573539733887)","(s_cubism-03.jpg, 0.10178208351135254)","(s_impressionist-05.jpg, 0.11082279682159424)","(s_cubism-04.jpg, 0.11676645278930664)","(s_cubism-08.jpg, 0.11799651384353638)","(s_cubism-14.jpg, 0.12990546226501465)"
1,"(s_cubism-02.jpg, 0.0)","(s_cubism-11.jpg, 0.07489609718322754)","(s_cubism-04.jpg, 0.10238879919052124)","(s_cubism-14.jpg, 0.10708343982696533)","(s_cubism-09.jpg, 0.10893464088439941)","(s_cubism-05.jpg, 0.11731451749801636)","(s_cubism-08.jpg, 0.13016170263290405)","(s_cubism-01.jpg, 0.13505542278289795)","(s_cubism-06.jpg, 0.13712078332901)","(s_impressionist-13.jpg, 0.14053857326507568)","(s_cubism-13.jpg, 0.14400577545166016)"
2,"(s_cubism-03.jpg, 0.0)","(s_cubism-01.jpg, 0.10178208351135254)","(s_cubism-05.jpg, 0.10908150672912598)","(s_impressionist-13.jpg, 0.10950326919555664)","(s_cubism-09.jpg, 0.1130605936050415)","(s_impressionist-12.jpg, 0.1225196123123169)","(s_cubism-08.jpg, 0.12502789497375488)","(s_cubism-16.jpg, 0.12599045038223267)","(s_impressionist-05.jpg, 0.12665539979934692)","(s_cubism-06.jpg, 0.13407009840011597)","(s_cubism-04.jpg, 0.14046704769134521)"
3,"(s_cubism-04.jpg, 0.0)","(s_cubism-09.jpg, 0.1018645167350769)","(s_cubism-02.jpg, 0.10238879919052124)","(s_impressionist-13.jpg, 0.10671412944793701)","(s_cubism-05.jpg, 0.10984539985656738)","(s_cubism-14.jpg, 0.11171084642410278)","(s_cubism-13.jpg, 0.1146041750907898)","(s_cubism-01.jpg, 0.11676645278930664)","(s_cubism-16.jpg, 0.11739301681518555)","(s_cubism-12.jpg, 0.11804777383804321)","(s_cubism-06.jpg, 0.11953151226043701)"
4,"(s_cubism-05.jpg, 0.0)","(s_impressionist-13.jpg, 0.07504183053970337)","(s_cubism-11.jpg, 0.07880634069442749)","(s_cubism-16.jpg, 0.08178341388702393)","(s_cubism-01.jpg, 0.085743248462677)","(s_cubism-08.jpg, 0.09565407037734985)","(s_cubism-09.jpg, 0.10248124599456787)","(s_impressionist-09.jpg, 0.10249888896942139)","(s_cubism-14.jpg, 0.10747575759887695)","(s_cubism-03.jpg, 0.10908150672912598)","(s_cubism-04.jpg, 0.10984539985656738)"
5,"(s_cubism-06.jpg, 0.0)","(s_cubism-13.jpg, 0.05005824565887451)","(s_cubism-09.jpg, 0.05835539102554321)","(s_impressionist-13.jpg, 0.0678321123123169)","(s_cubism-10.jpg, 0.07420200109481812)","(s_cubism-14.jpg, 0.07514625787734985)","(s_cubism-15.jpg, 0.09340512752532959)","(s_cubism-11.jpg, 0.099312424659729)","(s_cubism-16.jpg, 0.10892671346664429)","(s_cubism-04.jpg, 0.11953151226043701)","(s_impressionist-04.jpg, 0.12297338247299194)"
6,"(s_cubism-07.jpg, 0.0)","(s_cubism-16.jpg, 0.08965945243835449)","(s_cubism-15.jpg, 0.10735636949539185)","(s_impressionist-13.jpg, 0.11776244640350342)","(s_cubism-05.jpg, 0.12175154685974121)","(s_cubism-04.jpg, 0.1288130283355713)","(s_cubism-12.jpg, 0.15532207489013672)","(s_cubism-09.jpg, 0.16912662982940674)","(s_cubism-01.jpg, 0.17457890510559082)","(s_cubism-11.jpg, 0.17483818531036377)","(s_cubism-03.jpg, 0.1772039532661438)"
7,"(s_cubism-08.jpg, 0.0)","(s_cubism-05.jpg, 0.09565407037734985)","(s_cubism-01.jpg, 0.11799651384353638)","(s_cubism-03.jpg, 0.12502789497375488)","(s_cubism-02.jpg, 0.13016170263290405)","(s_impressionist-12.jpg, 0.1406012773513794)","(s_impressionist-09.jpg, 0.14667755365371704)","(s_cubism-04.jpg, 0.1479659080505371)","(s_cubism-09.jpg, 0.15091580152511597)","(s_impressionist-15.jpg, 0.15502697229385376)","(s_impressionist-05.jpg, 0.15538126230239868)"
8,"(s_cubism-09.jpg, 0.0)","(s_cubism-14.jpg, 0.049005746841430664)","(s_cubism-06.jpg, 0.05835539102554321)","(s_impressionist-08.jpg, 0.06291317939758301)","(s_impressionist-13.jpg, 0.06542342901229858)","(s_cubism-11.jpg, 0.0807008147239685)","(s_cubism-13.jpg, 0.08803653717041016)","(s_impressionist-04.jpg, 0.09485089778900146)","(s_cubism-15.jpg, 0.09553557634353638)","(s

In [13]:
#Returns the first element of any list/tuple
def func(sorted_neighbour_tuple):
    return sorted_neighbour_tuple[0]

In [14]:
cosine = imagename.applymap(func)


In [15]:
cosine.to_csv("Method1Git.csv")